### 1. 라이브러리, 데이터, 세션

In [1]:
import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType, BooleanType}
import org.apache.spark.sql.functions.{col, regexp_replace, explode, split, length}
import org.apache.spark.sql.Row
import spark.implicits._

Intitializing Scala interpreter ...

Spark Web UI available at http://218.38.137.27:4040
SparkContext available as 'sc' (version = 3.5.0, master = local[*], app id = local-1700732583806)
SparkSession available as 'spark'


import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType, BooleanType}
import org.apache.spark.sql.functions.{col, regexp_replace, explode, split, length}
import org.apache.spark.sql.Row
import spark.implicits._


In [2]:
val spark = SparkSession.builder.appName("spark-scala-test").getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3c583d86


In [3]:
val dir_path = "/Users/b06/Desktop/yeardream/medi-05/data/naverplace_meta"
val json_path = "/naverplace_meta_1.json"
val path = dir_path + json_path
val data = spark.read.option("multiline", "true").format("json").load(path)

dir_path: String = /Users/b06/Desktop/yeardream/medi-05/data/naverplace_meta
json_path: String = /naverplace_meta_1.json
path: String = /Users/b06/Desktop/yeardream/medi-05/data/naverplace_meta/naverplace_meta_1.json
data: org.apache.spark.sql.DataFrame = [BaseNaverBlog:betbetter: struct<__typename: string, categoryNo: string ... 1 more field>, BaseNaverBlog:bondiolsc: struct<__typename: string, categoryNo: string ... 1 more field> ... 469 more fields]


### 2. 전역 변수

In [4]:
val columns = data.columns

val hospitalBases = columns.filter(_.contains("HospitalBase"))

val targetColumns = Seq(
  "id",
  "name", 
  "road", 
  "reviewSettings", 
  "conveniences", 
  "keywords", 
  "phone", 
  "virtualPhone", 
  "naverBookingUrl", 
  "talktalkUrl", 
  "paymentInfo", 
  "homepages",
  "visitorReviewsTotal",
  "description",
  "Images"
)

val stringColumns = Seq(
  "id",
  "name", 
  "road",
  "phone",
  "virtualPhone",
  "naverBookingUrl",
  "talktalkUrl",
  "visitorReviewsTotal"
)

val descriptionColumns: Seq[String] = Seq(
  "id",
  "description"
)

val structColumns: Seq[String] = Seq(
  "id",
  "reviewSettings",
  "homepages"
)

val reviewKeywordColumns: Seq[String] = Seq(
  "id",
  "reviewSettings.keyword"
)

val homepagesColumns: Seq[String] = Seq(
  "id",
  "homepages.repr.url",
  "homepages.repr.type",
  "homepages.repr.isDeadUrl",
  "homepages.repr.landingUrl"
)

val conveniencesColumns: Seq[String] = Seq(
  "id",
  "conveniences"
)

val keywordsColumns: Seq[String] = Seq(
  "id",
  "keywords"
)

val paymentsColumns: Seq[String] = Seq(
  "id",
  "paymentInfo"
)

columns: Array[String] = Array(BaseNaverBlog:betbetter, BaseNaverBlog:bondiolsc, BaseNaverBlog:dainhani, BaseNaverBlog:kundaeclinic, BaseNaverBlog:memeetsworld, BaseNaverBlog:orot-hani, BaseNaverBlog:zzz0501, BusStation:104073, BusStation:104091, BusStation:104094, BusStation:104132, BusStation:104140, BusStation:104154, BusStation:104158, BusStation:104172, BusStation:104181, BusStation:104212, BusStation:104222, BusStation:104231, BusStation:104321, BusStation:104386, BusStation:104395, BusStation:104428, BusStation:104440, BusStation:104459, BusStation:104500, BusStation:104507, BusStation:104517, BusStation:104532, BusStation:104544, BusStation:104554, BusStation:104573, BusStation:104576, BusStation:104578, BusStation:104580, BusStation:104582, BusStation:104594, BusStation:104598,...


In [5]:
val stringColumnsSchema = StructType(Seq(
  StructField("id", StringType, true),
  StructField("name", StringType, true),
  StructField("road", StringType, true),
  StructField("phone", StringType, true),
  StructField("virtualPhone", StringType, true),
  StructField("naverBookingUrl", StringType, true),
  StructField("talktalkUrl", StringType, true),
  StructField("visitorReviewsTotal", IntegerType, true)
))

val descriptionColumnsSchema = StructType(Seq(
  StructField("id", StringType, true),
  StructField("description", StringType, true),
  StructField("description_length", IntegerType, true)
))

val reviewKeywordColumnsSchema = StructType(Seq(
  StructField("id", StringType, true),
  StructField("review_keyword", StringType, true)
))

val homepagesColumnsSchema = StructType(Seq(
  StructField("id", StringType, true),
  StructField("url", StringType, true),
  StructField("type", StringType, true),
  StructField("isDeadUrl", BooleanType, true),
  StructField("landingUrl", StringType, true)
))

val conveniencesColumnsSchema = StructType(Seq(
  StructField("id", StringType, true),
  StructField("conveniences", StringType, true)
))

val keywordsColumnsSchema = StructType(Seq(
  StructField("id", StringType, true),
  StructField("keywords", StringType, true)
))

val paymentsColumnsSchema = StructType(Seq(
  StructField("id", StringType, true),
  StructField("payments", StringType, true)
))

// 빈 데이터프레임 생성
var stringTable: DataFrame = spark.createDataFrame(spark.sparkContext.emptyRDD[Row], stringColumnsSchema)
var descriptionTable: DataFrame = spark.createDataFrame(spark.sparkContext.emptyRDD[Row], descriptionColumnsSchema)
var reviewKeywordTable: DataFrame = spark.createDataFrame(spark.sparkContext.emptyRDD[Row], reviewKeywordColumnsSchema)
var homepagesTable: DataFrame = spark.createDataFrame(spark.sparkContext.emptyRDD[Row], homepagesColumnsSchema)
var conveniencesTable: DataFrame = spark.createDataFrame(spark.sparkContext.emptyRDD[Row], conveniencesColumnsSchema)
var keywordsTable: DataFrame = spark.createDataFrame(spark.sparkContext.emptyRDD[Row], keywordsColumnsSchema)
var paymentsTable: DataFrame = spark.createDataFrame(spark.sparkContext.emptyRDD[Row], paymentsColumnsSchema)

stringColumnsSchema: org.apache.spark.sql.types.StructType = StructType(StructField(id,StringType,true),StructField(name,StringType,true),StructField(road,StringType,true),StructField(phone,StringType,true),StructField(virtualPhone,StringType,true),StructField(naverBookingUrl,StringType,true),StructField(talktalkUrl,StringType,true),StructField(visitorReviewsTotal,IntegerType,true))
descriptionColumnsSchema: org.apache.spark.sql.types.StructType = StructType(StructField(id,StringType,true),StructField(description,StringType,true),StructField(description_length,IntegerType,true))
reviewKeywordColumnsSchema: org.apache.spark.sql.types.StructType = StructType(StructField(id,StringType,true),StructField(review_keyword,StringType,true))
homepagesColumnsSchema: org.apache.spark.sql.types.Stru...


### 3. 함수

In [6]:
def getTable(df: DataFrame, columns: Seq[String], table: DataFrame): DataFrame = {
  val getColumns = df.select(columns.map(col): _*)
  val row = removeNull(getColumns)
  table.union(row)
}

def getStringTable(df: DataFrame, columns: Seq[String], table: DataFrame): DataFrame = {
  val getColumns = df.select(columns.map(col): _*)
  val row = removeNull(getColumns)
    .withColumn("road", regexp_replace(col("road"), "\n", ""))
  table.union(row)
}

def removeNull(df: DataFrame): DataFrame = {
  df.filter(col("name").isNotNull)
}

def getDescriptionTable(df: DataFrame, columns: Seq[String], table: DataFrame): DataFrame = {
  val cols = df.select(columns.map(col): _*)
  val rows = removeNull(cols)
    .withColumn("description", regexp_replace(col("description"), "\n", " "))
    .withColumn("description", regexp_replace(col("description"), "\r", " "))
    .withColumn("description", regexp_replace(col("description"), "\\*", ""))
    .withColumn("description", regexp_replace(col("description"), ",", " "))
    .withColumn("description_length", length(col("description")))
  table.union(rows)
}

def preprocessingReviewKeyword(reviewKeywordRow: DataFrame): DataFrame = {
  reviewKeywordRow
    .withColumnRenamed("keyword", "review_keyword")
    .withColumn("review_keyword", explode(split(col("review_keyword"), ", ")))
}

def getReviewKeywordTable(structDf: DataFrame, reviewKeywordColumns: Seq[String], reviewKeywordTable: DataFrame): DataFrame = {
  val getReviewKeywordColumns = structDf.select(reviewKeywordColumns.map(col): _*)
  val reviewKeywordRow = removeNull(getReviewKeywordColumns)
  val processedReviewKeywordRow = preprocessingReviewKeyword(reviewKeywordRow)
  reviewKeywordTable.union(processedReviewKeywordRow)
}

def checkNull(df: DataFrame, column: String): Boolean = {
  val cnt = df.filter(col(column).isNull).count()
  cnt == 25
}

def getHomepagesTable(structDf: DataFrame, homepagesColumns: Seq[String], homepagesTable: DataFrame): DataFrame = {
  if (checkNull(structDf, "homepages.repr")) {
    homepagesTable
  } else {
    getTable(structDf, homepagesColumns, homepagesTable)
  }
}

def getTableAndExplode(df: DataFrame, columns: Seq[String], table: DataFrame, column: String): DataFrame = {
  if (checkNull(df, column)) {
    table
  } else {
    val getColumns = df.select(columns.map(col): _*)
    val row = removeNull(getColumns)
    val rows = row.withColumn(column, explode(row(column)))
    table.union(rows)
  }
}

def saveToParquet(df: DataFrame, name: String): Unit = {
  val base_path = "/Users/b06/Desktop/yeardream/medi-05/data/output/"
  val path = base_path + name
  df.write.mode("append").parquet(path)
}

getTable: (df: org.apache.spark.sql.DataFrame, columns: Seq[String], table: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame
getStringTable: (df: org.apache.spark.sql.DataFrame, columns: Seq[String], table: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame
removeNull: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame
getDescriptionTable: (df: org.apache.spark.sql.DataFrame, columns: Seq[String], table: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame
preprocessingReviewKeyword: (reviewKeywordRow: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame
getReviewKeywordTable: (structDf: org.apache.spark.sql.DataFrame, reviewKeywordColumns: Seq[String], reviewKeywordTable: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataF...


### 4. 데이터 전처리

In [ ]:
hospitalBases.foreach { hospitalBase =>
    val hospitalBaseData = data.select(hospitalBase)
    val getColumns = targetColumns.map(t => col(s"$hospitalBase.$t").alias(t))
    val df = hospitalBaseData.select(getColumns: _*)
    val structDf = df.select(structColumns.map(col): _*)

    stringTable = getTable(df, stringColumns, stringTable)
    descriptionTable = getDescriptionTable(df, descriptionColumns, descriptionTable)
    reviewKeywordTable = getReviewKeywordTable(structDf, reviewKeywordColumns, reviewKeywordTable)
    // println(4)
    // homepagesTable = getHomepagesTable(structDf, homepagesColumns, homepagesTable)
    // println(5)
    // conveniencesTable = getTableAndExplode(df, conveniencesColumns, conveniencesTable, "conveniences")
    // println(6)
    // keywordsTable = getTableAndExplode(df, keywordsColumns, keywordsTable, "keywords")
    // println(7)
    // paymentsTable = getTableAndExplode(df, paymentsColumns, paymentsTable, "paymentInfo")
}

1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4


In [ ]:
stringTable.show(10)
descriptionTable.show(10)
reviewKeywordTable.show(10)

In [ ]:
// 데이터 저장
saveToParquet(stringTable, "string_table")
saveToParquet(descriptionTable, "description_table")
saveToParquet(reviewKeywordTable, "review_keyword_table")
saveToParquet(homepagesTable, "homepages_table")
saveToParquet(conveniencesTable, "conveniences_table")
saveToParquet(keywordsTable, "keywords_table")
saveToParquet(paymentsTable, "payments_table")